In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [2]:
alg_list = ["fraudar", "rsd", "rev2", "sg"][:]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [1000.0, 2000.0, 3000.0, 4000.0]
frac_list = [0.0, 0.2, 0.4, 0.6, 0.8]

In [3]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = {
            a: dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
            for a in alg_list
        }
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({(a, sp): {u: scores[sp][a][u] for u in scores[sp][a]} for a in alg_list for sp in scores if a in scores[sp]})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], (slice(None), "truth")] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], (slice(None), "truth")] = 1
    return df

In [4]:
def get_metrics(df_res, splits=4, K=None, th_dict=None):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        # df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        # df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        for a in alg_list:
            if (a, sp) not in df_res.columns:
                continue
            df = df_res[[(a, sp), (a, "truth")]].dropna()
            t = df[(a, "truth")].sum() if K is None else K
            t_frac = t / df.shape[0]
            # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
            # df["pred"] = df[sp] >= t_frac
            df[(a, "pred")] = df[(a, sp)] >= th_dict[a]

            ret[(a, sp)] = {
                "precision": sklearn.metrics.precision_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")]),
                "recall": sklearn.metrics.recall_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")]),
                "f1": sklearn.metrics.f1_score(y_true=df[(a, "truth")], y_pred=df[(a, "pred")])
            }
    return ret

In [5]:
attack_res = {
    (d, a, b): get_results(data_name=d, alg=a, attack_name="sockfarm_large_single", budget=b)
    for d in data_list for a in alg_list
    for b in budget_list
}

../res/sockfarm_large_single/sg-otc/3000.0.pkl
../res/sockfarm_large_single/sg-otc/4000.0.pkl


In [6]:
# attack_res[("amazon", "rev2", 400.0)].to_csv("ar.csv")

In [9]:
def get_sockfarm_alg_data_met(df_met, data="alpha", met="f1"):
    rdf = df_met.reset_index().pivot(index=["data", "algo", "metrics"], columns=["budget"]).loc[(data, slice(None), met)].droplevel([0, 2], axis=0)
    rdf.index = [f"sockfarm-{i}" for i in rdf.index]
    return rdf

In [31]:
def_dict = {
    "fraudar": {"fraudar": 0.9, "rsd": 1.96, "rev2": 0.028, "sg": 0.5}, # rev2=0.026
    "rsd": {"fraudar": 0.5, "rsd": 0.51, "rev2": 0.026, "sg": 0.5},
    "rev2": {"fraudar": 0.6, "rsd": 1.01, "rev2": 0.5, "sg": 0.5},
    "sg": {"fraudar": 0.9, "rsd": 1.4, "rev2": 0.15, "sg": 0.5},
}


dict_met = {
    (d, a, b): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b)], splits=4, K=None, th_dict=def_dict[a]))
    for d, a, b in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
df_met.index = df_met.index.set_names(["data", "algo", "budget", "metrics"])
df_met.columns = pd.MultiIndex.from_tuples(df_met.columns)
df_met = df_met.mean(axis=1, level=0)
# display(df_met)

dmet="otc"

display(get_sockfarm_alg_data_met(df_met, data=dmet, met="f1"))
display(get_sockfarm_alg_data_met(df_met, data=dmet, met="recall"))
display(get_sockfarm_alg_data_met(df_met, data=dmet, met="precision"))

fraudar                                     rsd            \
budget              1000.0    2000.0    3000.0    4000.0    1000.0    2000.0   
sockfarm-fraudar  0.418383  0.323221  0.264050  0.223373  0.234432  0.160007   
sockfarm-rev2     0.418383  0.323221  0.265391  0.222822  0.403089  0.285859   
sockfarm-rsd      0.418383  0.323221  0.265391  0.223351  0.401127  0.284167   
sockfarm-sg       0.418383  0.323964       NaN       NaN  0.339241  0.237810   

                                          rev2                                \
budget              3000.0    4000.0    1000.0    2000.0    3000.0    4000.0   
sockfarm-fraudar  0.121455  0.097874  0.467400  0.365759  0.300374  0.254843   
sockfarm-rev2     0.226433  0.187596  0.273632  0.190297  0.144890  0.115988   
sockfarm-rsd      0.220025  0.179510  0.470105  0.367548  0.301676  0.255209   
sockfarm-sg            NaN       NaN  0.092003  0.061253       NaN       NaN   

                        sg                                
budget              1000.0    2000.0    3000.0    4000.0  
sockfarm-fraudar  0.070794  0.050036  0.038695  0.031531  
sockfarm-rev2     0.070794  0.050036  0.038695  0.031531  
sockfarm-rsd      0.070794  0.050036  0.038695  0.031531  
sockfarm-sg       0.070794  0.050036       NaN       NaN

fraudar                                     rsd            \
budget              1000.0    2000.0    3000.0    4000.0    1000.0    2000.0   
sockfarm-fraudar  0.374817  0.245575  0.183157  0.146201  0.132803  0.086967   
sockfarm-rev2     0.374817  0.245575  0.184232  0.145780  0.259042  0.169630   
sockfarm-rsd      0.374817  0.245575  0.184232  0.146198  0.256901  0.168216   
sockfarm-sg       0.374817  0.246279       NaN       NaN  0.206256  0.135811   

                                          rev2                                \
budget              3000.0    4000.0    1000.0    2000.0    3000.0    4000.0   
sockfarm-fraudar  0.064656  0.051457  0.442611  0.290027  0.215688  0.171686   
sockfarm-rev2     0.129299  0.104573  0.164636  0.107812  0.079610  0.062509   
sockfarm-rsd      0.125055  0.099522  0.448064  0.292901  0.217282  0.172106   
sockfarm-sg            NaN       NaN  0.048278  0.031618       NaN       NaN   

                        sg                               
budget              1000.0    2000.0    3000.0   4000.0  
sockfarm-fraudar  0.044424  0.029223  0.021773  0.01735  
sockfarm-rev2     0.044424  0.029223  0.021773  0.01735  
sockfarm-rsd      0.044424  0.029223  0.021773  0.01735  
sockfarm-sg       0.044424  0.029223       NaN      NaN

fraudar                                     rsd            \
budget              1000.0    2000.0    3000.0    4000.0    1000.0    2000.0   
sockfarm-fraudar  0.475059  0.475059  0.475765  0.476458  1.000000  1.000000   
sockfarm-rev2     0.475059  0.475059  0.477153  0.475785  0.908429  0.908429   
sockfarm-rsd      0.475059  0.475059  0.477153  0.476398  0.915009  0.915009   
sockfarm-sg       0.475059  0.475732       NaN       NaN  0.955727  0.955842   

                                          rev2                                \
budget              3000.0    4000.0    1000.0    2000.0    3000.0    4000.0   
sockfarm-fraudar  1.000000  1.000000  0.496576  0.497131  0.497131  0.497131   
sockfarm-rev2     0.910788  0.912444  0.810898  0.810898  0.805579  0.803451   
sockfarm-rsd      0.915009  0.915009  0.495779  0.495210  0.495946  0.496642   
sockfarm-sg            NaN       NaN  0.979167  0.979167       NaN       NaN   

                        sg                                
budget              1000.0    2000.0    3000.0    4000.0  
sockfarm-fraudar  0.175793  0.175793  0.175793  0.175047  
sockfarm-rev2     0.175793  0.175793  0.175793  0.175047  
sockfarm-rsd      0.175793  0.175793  0.175793  0.175047  
sockfarm-sg       0.175793  0.175793       NaN       NaN